In [191]:
from elasticsearch import Elasticsearch, helpers, AsyncElasticsearch
import numpy as np
import glob
import json

In [169]:
def check_ctx_len(data_list: list):
    prag_list = [] # 길이 정보만 
    check_docs = []

    for doc in data_list:
        form = {
        "title":"",
        "len_pragraphs":0,
        "len_ctxs": 0,
        "len_qas": 0,
        "context":""
        }
        try:
            form["title"] = doc['title']
        except:
            form["title"] = doc['doc_title']
        form['len_pragraphs'] = len(doc['paragraphs'])
        for prag in doc['paragraphs']:
            form["len_ctxs"] = len(prag['context'].split(" "))
            form["context"] = prag['context']
            form["len_qas"] = len(prag["qas"])
        prag_list.append(form)
    return prag_list
    

In [204]:
def show_doc_len(res, func):
    res = check_ctx_len(res)# 최대 최소 등 길이 정보를 조회하는 목적
    idx = -1
    len_list = [l["len_ctxs"] for l in res]
    print(func(len_list))
    try:
        idx = len_list.index(func(len_list))
    except:
        pass
   #return res, idx

In [63]:
def index_mrc_docs(idx: str, docs: dict, title: str = 'title', doc_id: str = None) -> list:
    # title 키값 이름이 다달라서 파라메터로 받음
    docs_insert = []
    index = idx
    for k, v in docs.items():
        tmp = {}
        src = f"{index}_{k}"
        tmp["_index"] = index
        tmp["source"] = src
        
        for d in v['data']:
            tmp["title"] = d[title]
            if doc_id:
                tmp["doc_id"] = d[doc_id]
            for single_prag in d["paragraphs"]:
                tmp.update(single_prag)
                docs_insert.append(tmp)
    return docs_insert

In [42]:
def load_dataset(path_list: list, train: str = 'train') :
    # train 데이터 파일 이름에 train이 없을때를 대비하려고 파라메터로 받음
    docs = {}
    for doc in path_list:
        with open( doc, "r",encoding='utf8') as fr:
            if train in doc:
                docs['train'] = json.load(fr)
            else:
                docs['dev'] = json.load(fr)
    return docs

In [3]:
es_config = glob.glob("/home/hjb/workspace/elk_practice/resources/es_idx_config/mrc/*")
print(es_config)

['/home/hjb/workspace/elk_practice/resources/es_idx_config/mrc/mrc_mapping.json', '/home/hjb/workspace/elk_practice/resources/es_idx_config/mrc/setting.json']


# Create Index

In [4]:
payload = {}

for config_ in es_config:
    with open(config_, 'r', encoding='utf8') as fr:
        if 'mapping' in config_:
            payload.update(json.load(fr))
        else:
            payload.update(json.load(fr))

In [7]:
es = Elasticsearch(["http://localhost:9200"])


In [8]:
es.info()

ObjectApiResponse({'name': 'e9536fb1b1a5', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'FAWHVAD6QMO9equB10N1SQ', 'version': {'number': '8.0.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '801d9ccc7c2ee0f2cb121bbe22ab5af77a902372', 'build_date': '2022-02-24T13:55:40.601285296Z', 'build_snapshot': False, 'lucene_version': '9.0.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
indices=["admin_mrc","book_mrc", "news_mrc", "minds_news_mrc", "korquad1"]


In [111]:
es.indices.delete(index="mindslab_news_mrc" )

ObjectApiResponse({'acknowledged': True})

In [38]:
for index in indices:
    try:
        #es.indices.delete(index=index)
        es.indices.create(index=index ,body=payload)
    except Exception:
        pass

/tmp/ipykernel_14755/713483641.py:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index=index ,body=payload)


In [109]:
res = es.cat.indices(format="json", v=True)
for r in res: 
    print(r)

{'health': 'yellow', 'status': 'open', 'index': 'korquad1', 'uuid': '5KJ9IFgpTaGGIQ--aSBjWg', 'pri': '2', 'rep': '1', 'docs.count': '10645', 'docs.deleted': '0', 'store.size': '22.9mb', 'pri.store.size': '22.9mb'}
{'health': 'yellow', 'status': 'open', 'index': 'mindslab_news_mrc', 'uuid': '_bcMDqXhQL2E2J9OyT0CCQ', 'pri': '1', 'rep': '1', 'docs.count': '47314', 'docs.deleted': '0', 'store.size': '45.9mb', 'pri.store.size': '45.9mb'}
{'health': 'yellow', 'status': 'open', 'index': 'book_mrc', 'uuid': 'dK9nEP6gROeTvvgoX7-MYA', 'pri': '2', 'rep': '1', 'docs.count': '237500', 'docs.deleted': '0', 'store.size': '476.4mb', 'pri.store.size': '476.4mb'}
{'health': 'yellow', 'status': 'open', 'index': 'news_mrc', 'uuid': '-DOy6kTORkiD_HoL8Fh_fg', 'pri': '2', 'rep': '1', 'docs.count': '125964', 'docs.deleted': '0', 'store.size': '407.7mb', 'pri.store.size': '407.7mb'}
{'health': 'yellow', 'status': 'open', 'index': 'admin_mrc', 'uuid': 'MKb3eAXgR3qziQrLwLrK8g', 'pri': '2', 'rep': '1', 'docs.coun

# Korquad Indexing

In [186]:
kor_list = glob.glob("./raw_korquad/*.json")

In [187]:
kor_list

['./raw_korquad/KorQuAD_v1.0_train.json',
 './raw_korquad/KorQuAD_v1.0_dev.json']

In [188]:
docs = load_dataset(kor_list)

In [189]:
docs['dev']['data'][0].keys()

dict_keys(['paragraphs', 'title'])

In [206]:
print("** mean **")
show_doc_len(docs['train']['data'], np.mean)
show_doc_len(docs['dev']['data'], np.mean)
print("** median **")
show_doc_len(docs['train']['data'], np.median)
show_doc_len(docs['dev']['data'], np.median)
print("** max **")
show_doc_len(docs['train']['data'], max)
show_doc_len(docs['dev']['data'], max)

** mean **
109.99718309859155
114.7
** median **
100.0
100.0
** max **
482
523


## korquad 1.0 최대 단락 길이

523

In [48]:
insert_docs = index_mrc_docs(idx='korquad1', docs=docs, title='title')
print(len(insert_docs))

10645


In [49]:
helpers.bulk(es, insert_docs)

(10645, [])

# AIHUB data
## admin_mrc

In [207]:
aihub_list = glob.glob("./aihub/admin_mrc/**/*span*.json")


In [208]:
aihub_list

['./aihub/admin_mrc/train/TL_span_extraction_how.json',
 './aihub/admin_mrc/train/TL_span_extraction.json',
 './aihub/admin_mrc/dev/VL_span_extraction_how.json',
 './aihub/admin_mrc/dev/VL_span_extraction.json']

In [209]:
docs = load_dataset(aihub_list, 'TL')

In [166]:
docs['train']['data'][0].keys()

dict_keys(['doc_id', 'doc_title', 'doc_source', 'doc_published', 'doc_class', 'created', 'paragraphs'])

In [210]:
print("** mean **")
show_doc_len(docs['train']['data'], np.mean)
show_doc_len(docs['dev']['data'], np.mean)
print("** median **")
show_doc_len(docs['train']['data'], np.median)
show_doc_len(docs['dev']['data'], np.median)
print("** max **")
show_doc_len(docs['train']['data'], max)
show_doc_len(docs['dev']['data'], max)

** mean **
125.46296064568604
128.36695869837297
** median **
121.0
123.0
** max **
431
378


In [70]:
insert_docs = index_mrc_docs(idx='admin_mrc', docs=docs, title='doc_title', doc_id='doc_id')

In [72]:
helpers.bulk(es, insert_docs)

(75917, [])

## news_mrc

In [211]:
aihub_list = glob.glob("./aihub/news_mrc/**/*extract*.json")

In [212]:
aihub_list

['./aihub/news_mrc/train/TL_span_extraction.json',
 './aihub/news_mrc/dev/VL_span_extraction.json']

In [213]:
docs = load_dataset(aihub_list, 'train')

In [76]:
docs['train']['data'][0].keys()

dict_keys(['doc_id', 'doc_title', 'doc_source', 'doc_published', 'doc_class', 'created', 'paragraphs'])

In [214]:
print("** mean **")
show_doc_len(docs['train']['data'], np.mean)
show_doc_len(docs['dev']['data'], np.mean)
print("** median **")
show_doc_len(docs['train']['data'], np.median)
show_doc_len(docs['dev']['data'], np.median)
print("** max **")
show_doc_len(docs['train']['data'], max)
show_doc_len(docs['dev']['data'], max)

** mean **
170.34916537908492
230.71208116024863
** median **
153.0
198.0
** max **
495
486


In [77]:
insert_docs = index_mrc_docs(idx='news_mrc', docs=docs, title='doc_title', doc_id='doc_id')

In [78]:
insert_docs[0]

{'_index': 'news_mrc',
 'source': 'news_mrc_train',
 'title': '광주대 장기근속자·스승의날 유공 장관 표창 전수식',
 'doc_id': '01600801.20210610140606002',
 'context': '광주대학교(총장 김혁종)는 지난 9일 학교 행정관 회의실에서 장기근속자 공로패 수여식 및 스승의날 유공 교원 교육부장관 표창 전수식을 개최했다.\n10일 광주대에 따르면 학교법인 호심학원 창설 51주년·광주대학교 개교 41주년을 기념해 진행된 이번 장기근속자 공로패 수여 대상자는 30년 근속 이봉규 인테리어디자인학과 교수와 박우미 패션·주일리디자인학부 교수 등 6명이다.\n이날 제40회 스승의날 유공 교원 교육부장관 표창 전수식도 함께 열렸다. 교육부장관 표창은 올바른 교육관과 투철한 사명감으로 사도를 실천하고 헌신적인 노력을 통해 교육 발전에 이바지한 교원에게 수여된다.\n김덕현 보건행정학부 교수와 임형택 청소년상담·평생교육학과 교수가 인재 양성 분야에서 수상의 영예를 안았다.\n김 교수는 연구와 실무경험으로 대학과 지역 산업계의 유대를 강화하고 이를 바탕으로 인재 양성에 기여한 공로를 인정받았다.\n또 임 교수는 다년간의 청소년 교육 분야 연구를 바탕으로 산학연계를 통한 맞춤형 교육과 후학양성에 노력한 점 등이 높은 평가를 받았다.',
 'qas': [{'qa_type': 1,
   'question_id': 4830354,
   'question': '광주대학교로부터 공로패를 전달받게 된 교원은 몇 명이야',
   'is_impossible': False,
   'answers': {'text': '6명',
    'answer_start': 202,
    'clue_start': None,
    'clue_text': None,
    'options': None}},
  {'qa_type': 1,
   'question_id': 4961424,
   'question': '광주대학교로부터 

In [79]:
helpers.bulk(es, insert_docs)

(125964, [])

## book_mrc

In [215]:
aihub_list = glob.glob("./aihub/book_mrc/*.json")

In [216]:
aihub_list

['./aihub/book_mrc/book_dev.json', './aihub/book_mrc/book_train.json']

In [217]:
docs = load_dataset(aihub_list)

In [218]:
print("** mean **")
show_doc_len(docs['train']['data'], np.mean)
show_doc_len(docs['dev']['data'], np.mean)
print("** median **")
show_doc_len(docs['train']['data'], np.median)
show_doc_len(docs['dev']['data'], np.median)
print("** max **")
show_doc_len(docs['train']['data'], max)
show_doc_len(docs['dev']['data'], max)

** mean **
111.38915797317436
112.23570712136409
** median **
110.0
112.0
** max **
170
171


In [134]:
max([ l["len_ctxs"] for l in res])

171

최대 길이 171. 아주 긴 단락은 없음

In [83]:
docs['train']['data'][0].keys()

dict_keys(['time', 'title', 'agency', 'year', 'content_id', 'KDC', 'paragraphs'])

In [84]:
insert_docs = index_mrc_docs(idx='book_mrc', docs=docs, title='title', doc_id='content_id')
print(len(insert_docs))

237500


In [85]:
helpers.bulk(es, insert_docs)

(237500, [])

## minds_news_mrc

In [220]:
aihub_list = glob.glob("./aihub/mindslab_news_mrc/*.json")

In [221]:
aihub_list

['./aihub/mindslab_news_mrc/ko_nia_noanswer_squad_all.json',
 './aihub/mindslab_news_mrc/ko_nia_clue0529_squad_all.json',
 './aihub/mindslab_news_mrc/ko_nia_normal_squad_all.json']

In [222]:
docs = load_dataset(aihub_list)

In [223]:
docs['dev']["data"][40001].keys()

dict_keys(['title', 'paragraphs', 'source'])

In [224]:
print("** mean **")
show_doc_len(docs['dev']['data'], np.mean)
print("** median **")
show_doc_len(docs['dev']['data'], np.median)
print("** max **")
show_doc_len(docs['dev']['data'], max)

** mean **
196.71443970072283
** median **
180.0
** max **
1481


In [120]:
insert_docs = index_mrc_docs(idx='minds_news_mrc', docs=docs, title='title', doc_id=False)
print(len(insert_docs))

47314


In [123]:
helpers.bulk(es, insert_docs)

(47314, [])

In [1]:
print( 75917 + 125964 + 237500 + 47314 )

486695


# 간단 요약 노트

총 단락 수는 486,695.  
  
단락 별 길이가 대략 100 언저리(110~190).  
최대길이는 1,481(Mindslab이 구축한 뉴스 MRC 데이터)  
긴 단락들은 보통 300~500  

---  

#### 긴 단락 처리 잔략

**처리 안했을 때랑 했을 때 결과 차이 비교를 해야함**

1. 100으로 자른다고 했을 때, 남은 단어가 너무 짧은 경우(10 단어 이하) 버림.
2. 잘라내서 정답이 영역이 삭제된 경우는 체크 필요.
3. 영억이 사라지지 않은 경우, offset 변화량은 체크해야함.  

분리되서 추가된 단락인 경우 원래 전체 단락 형태, 그 단락에서 원래 offset 정보를 가지고 있어야함. parent_id, origin_start_offset 기억해서 더해주면 될 것.
  
#### positive, negative ctxs들 처리 방법
  
* 검색 영역: title, ctxs
* 검색 쿼리: 정답 영역 단어들 + question 
* 오답 정의: 답이 passage 안에 없음
  
1. 정답 걸려 나온 경우 -> pos_ctxs 리스트 포함( 최대 100개? )
2. 오답들 -> score가 0인 것들(타 단락에서 정답인 것으로 in batch 한다면 내가 고려할 건 없는 게 아닌지?)
3. hard negative -> score가 0이 아닌데 오답임 == answer 는 match 안되고 question만 match 됨.